In [ ]:
import cv2
import os
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.model_selection import train_test_split
import bm3d
from skimage import io, img_as_float
import matplotlib.pyplot as plt

# for denoising:
from skimage.restoration import denoise_wavelet, estimate_sigma
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio



: 

work onn VISION dataset:
work on DO1 camera only

CALCULATING THE PRNU FOR A SPECIFIC CAMERA:
1. get all flat images in one folder - Sali
2. convert flat images to grayscale - Sali
3. denoising (using wavelet) - Aditya 
4. calculate noise residual (original - denoise image) for each image in the flat images - Aditya
5. calculate: equation in image "eq1' - Sali
6. calculate CRLB  in image 'eq2' - Sali

7. calculate linear pattern in image 'eq3', this is K which is the PRNU - write the equation in code format - Kavan
8. calculate conversion from RGB to grayscale in image 'eq4' or explore other conversion methods - Kavan

CALCULATING THE SIMILARITY BETWEEN A CAMERA'S FINGERPRINT AND A NEW TEST IMAGE:
work with natural images for camera DO1
9. calculate the noise residual (original image - denoised image)  (W2) noise residual for test image - Sri

#########################################

10. calculate X and Y in equations in img 'eq5'
11. Calculate the NCC in img 'eq6'
12. calculate the PCE in img 'eq7'



In [2]:
data_dir = '/data/Sali/camera_fingerprint/Dresden_Exp'
camera_ls = os.listdir(data_dir)
camera_ls = camera_ls[:1]

# Create list of all camera directories
camera_dirs = [data_dir + '/' + camera for camera in camera_ls ]
camera_dirs


['/data/Sali/camera_fingerprint/Dresden_Exp/Agfa_DC-504_0']

### Moving images to folder

In [ ]:
images_output_dir = '/data/Sali/camera_fingerprint/Vision_dataset_organized'

In [10]:
import shutil

In [3]:
# def move_images_to_folder(camera_dir, output_dir):
#     list_of_cameras = [f'D{str(i).zfill(2)}' for i in range(1, 37)]
#     subdirs = ['flat', 'nat', 'natFBH', 'natFBL', 'natWA']

#     for cam in list_of_cameras:
        
#         cam_dir = os.path.join(output_dir, cam)
#         os.makedirs(cam_dir, exist_ok=True)  
#         subdirs = ['flat', 'nat', 'natFBH', 'natFBL', 'natWA']

#         for sub in subdirs:
#             sub_dir = os.path.join(cam_dir, sub)
#             os.makedirs(sub_dir, exist_ok=True)

#     for image in os.listdir(camera_dir):
#         if image.endswith('.jpg') or image.endswith('.png'):
#             image_parts = image.split('_')

#             camera_id = image_parts[0]
#             subdir = image_parts[2]
#             current_img_path = os.path.join(camera_dir, image)
#             new_img_path = os.path.join(output_dir, camera_id, subdir, image)

#             try:
#                 shutil.copy(current_img_path, new_img_path)
#             except FileNotFoundError:
#                 print(f"Error: {current_img_path} not found.")
#             except Exception as e:
#                 print(f"Error copying {image}: {e}")
    
# move_images_to_folder('/data/Sali/camera_fingerprint/', images_output_dir)

In [ ]:
# equation 1

def estimate_prnu(W_list, I_list):
    """
    Compute MLE of PRNU (Equation 6)
    W_list: list of noise residuals (numpy arrays)
    I_list: list of corresponding intensity images (numpy arrays)
    Returns: PRNU fingerprint estimate (same shape as input images)
    """
    numerator = np.zeros_like(W_list[0])
    denominator = np.zeros_like(I_list[0])

    for Wk, Ik in zip(W_list, I_list):
        numerator += Wk * Ik
        denominator += Ik * Ik

    K_hat = numerator / denominator
    return K_hat




In [ ]:
# equation 2

def prnu_crlb(I_list, sigma2):
    """
    Compute Cramér-Rao Lower Bound (Equation 7)
    I_list: list of intensity images (numpy arrays)
    sigma2: noise variance (scalar or array)
    Returns: lower bound on variance of PRNU estimate
    """
    denominator = np.zeros_like(I_list[0])

    for Ik in I_list:
        denominator += Ik * Ik

    var_bound = sigma2 / denominator
    return var_bound


### Extracting PRNU

In [ ]:
https://github.com/samuelebortolotti/neural-prnu-extractor #code to replicate
https://github.com/ocrim1996/prnu-python?tab=readme-ov-file
https://ieeexplore.ieee.org/document/7791195 # image alignment 
https://ieeexplore.ieee.org/document/7791195 # paper that talks about denoising filter
https://paperswithcode.com/sota/image-denoising-on-sidd # papers with code
https://www.reddit.com/r/AirlinerAbduction2014/comments/1fnxy5o/photo_response_nonuniformity_prnu_authentication/#:~:text=Extracting%20the%20PRNU%20requires%20denoising,is%20compared%20to%20the%20CRP. # reddit page

SyntaxError: invalid syntax (4290831171.py, line 1)